# 八进制移动轨迹

要想输出移动的轨迹，首先就要想清楚以下几个问题

1. 需要记录什么数据
2. 用什么数据结构来记录数据
  
首先我们肯定要记录**到过的坐标点**，**坐标点的状态**，并且可以看到输出的是一个矩阵的形式，所以我们需要记录**上下左右边界坐标的值**。

那么坐标点我们可以用`List`和`Tuple`来记录，坐标点的状态可以用`Dictionary`，上下边界坐标就简单地用 4 个变量就好了。

接下来我们就需要思考：当我接收到一串八进制 number 的时候，从这个 number 到最后的矩阵需要那些过程？

1. 当我们接收到一串八进制 number，需要把这串 number 分解为一个一个 digits，然后逐个处理这些 digits。所有 digits 处理完之后才进行打印。于是有了下面的 `Move` 函数。到这里我们发现，我们不能一次性处理完所有digits，一次只能处理一个，那么就还需要记录**当前位置**的数据，我们用一个 `Tuple` 就好了，于是就有了下面的`curr_pos`变量。
2. 现在我们需要处理每个 digit，我们就需要先判断这个 digit 表明的是哪个方位，才能有进一步操作，于是有了 `MoveOneStep` 函数。
3. `MoveOneStep` 判断了方位之后，就要 update `curr_pos`，然后把 updated `curr_pos` 添加到**到过的坐标点**、**坐标点的状态**里面去。并且还需要看看 updated `curr_pos` 是否有超过目前的边界，如果有，我们就需要 update **上下左右边界坐标的值**。于是就有了 `AddPoint` 函数。
4. 到此，数据记录所需的函数已经全部具备了，即 `Move`、 `MoveOneStep`、 `AddPoint`。
5. 把所有数据解析并记录下来之后，我们就能开始设计打印的函数了。即下面的 `PrintLocus` 函数。
6. 既然要打印矩阵，我们就需要有一个矩阵，我们可以用 `List` of `List`s，于是就有了 `PrintLocus` 函数里定义的 `graph`。外层的 `List` 的每个元素是一个 `List`，表示一行。内层的 `List` 的每个元素是一个 character，表示这一行里面的一个符号。
7. 我们已经知道了 x 和 y 的最大/小值，但因为打印的时候是打印一个矩阵，也就是整个矩形里的每个点都需要判断一次，因此我们就需要遍历 $x \in [x_{min}, x_{max}]$ 和 $y \in [y_{min}, y_{max}]$ 的每一种组合。所以我定义了 `x_span` 和 `y_span` 两个 list，然后用一个双层 `for` 循环来进行遍历。其实也可以直接用一个双层 `while` 循环，这样就不需要那两个 list 了。
8. 遍历每个点的时候，我们就需要判断这个点的状态了，是 on/off/unvisited 哪一种，然后根据状态来设置这个点对应的符号。
9. 所有点遍历完之后，因为示例输出的上下左右是反的，也就是旋转 180 度，所以我们需要把每一行都进行 reverse，这样左右就反过来了。又因为 print 的时候是从上往下 print 的，因此上下是默认反过来的。
10. 最后我们把每一行 list 都转成一个 string，然后 print 这个 string，就得到了想要的输出了。

In [1]:
class Locus:
    
    def __init__(self):
        self.point_locus = [(0, 0)]
        self.point_status = {(0, 0): True}
        self.x_max, self.x_min, self.y_max, self.y_min = 0, 0, 0, 0
        self.curr_pos = (0, 0)    
        
    def AddPoint(self, point):
        self.point_locus.append(point)
        
        if point in self.point_status:
            self.point_status[point] = not self.point_status[point]
        else :
            self.point_status[point] = True

        x, y = point
        if x > self.x_max:
            self.x_max = x
        elif x < self.x_min:
            self.x_min = x
        if y > self.y_max:
            self.y_max = y
        elif y < self.y_min:
            self.y_min = y

    def MoveOneStep(self, direction):
        x, y = self.curr_pos
        if direction == 0:
            self.curr_pos = (x, y+1)
        elif direction == 1:
            self.curr_pos = (x+1, y+1)
        elif direction == 2:
            self.curr_pos = (x+1, y)
        elif direction == 3:
            self.curr_pos = (x+1, y-1)
        elif direction == 4:
            self.curr_pos = (x, y-1)
        elif direction == 5:
            self.curr_pos = (x-1, y-1)
        elif direction == 6:
            self.curr_pos = (x-1, y)
        elif direction == 7:
            self.curr_pos = (x-1, y+1)
        else:
            print("Invalid direction of [{}]!".format(direction))
        
        self.AddPoint(self.curr_pos)
    
    def PrintLocus(self):
        graph = []
        x_span = [self.x_min+i for i in range(self.x_max-self.x_min+1)]
        y_span = [self.y_min+i for i in range(self.y_max-self.y_min+1)]
        for y in y_span:
            this_row = []
            for x in x_span:
                if (x, y) in self.point_status:
                    if self.point_status[(x, y)]:
                        this_row.append('o')
                    else:
                        this_row.append('x')  # 示例输出并没有区分 switch off 和 unvisited 的状态，我在这里区分一下
                else: this_row.append('*')
            graph.append(this_row)
        # 例子中的输出的上下左右是相反的，也就是整张图旋转了180度
        # graph.reverse()  # 这一步其实是不需要的，因为 print 的时候就是从上往下 print ，但是左右方向还是需要 reverse 的
        for i in range(len(graph)):
            graph[i].reverse()
        # 输出
        for row in graph:
            print(''.join(row))

    def Move(self, steps):
        self.__init__()  # 这一条语句会让每次 Move 的时候都清空旧数据并重新开始，如需保留旧数据就把这行注释掉。
        for step in steps:  # 逐个处理 digits
            self.MoveOneStep(int(step))
        # 所有 digits 处理完之后就可以打印
        print('='*50)
        print(steps)
        print('-'*20)
        self.PrintLocus()


In [2]:
locus = Locus()
locus.Move('1423701614')
locus.Move('35307402')

1423701614
--------------------
o***
*xoo
*oo*
xo**
o***
35307402
--------------------
o*
ox
xx
*o


到此，程序已经能够实现打印轨迹的功能了。

此外，我们还能发现，在 `class Locus` 里面定义的 `point_locus` 其实并没有派上什么用场，所以其实对于这道题它是可以被删除的。

# 集合

## Problem 1

|a|b|c|
|-|-|-|
|0|0|0|
|0|0|1|
|...|...|...|
|1|1|0|
|1|1|1|

c.i. Each value in A can map to one of $n$ values in B.

c.ii. There are ___ pairs such that one of the elements is drawn from A and the other from B (the order matters). Each pair may be in or not in a relation.

c.iii. 

## Problem 2

e. 

f. 

## Problem 3

b.

c. “\*” 运算符只牵扯到**补集**和**并集**。想要得到**空集**，我们可以取**全集**的**补集**。那么我们就现在就需要得到全集。

d.

